In [1]:
import pandas as pd
import numpy as np

# cac feature duoc su dung
features2 = ['SLHC', 'HST', 'HCT', 'MCV', 'MCH', 'MCHC', 'RDWCV', 'SLTC', 'SLBC', 'FE', 'FERRITIN', 'HBA1', 'HBA2', 'HBE', 'HBF']
features = ['SLHC', 'HST', 'HCT', 'MCV', 'MCH', 'MCHC', 'RDWCV', 'SLTC', 'SLBC']

In [2]:
df1 = pd.read_csv("../datasets/processed_data.csv", low_memory=False)
# chuyen PID ve KDL string
df1.PID = df1.PID.astype(str)
# cho PID lam index trong dataframe
df1.set_index("PID", inplace=True)
# lay ra cac records tuong ung voi cac features
feature_df = df1[features]
feature_df = feature_df.apply(pd.to_numeric, errors='coerce', downcast='float')
df1 = df1.loc[feature_df.dropna().index][features2 + ['sheet']]
condition = df1.sheet.isin(['alpha', 'fe_alpha', 'alpha_beta'])
df1['alpha'] = condition.astype(int)

condition = df1.sheet.isin(['beta', 'fe_beta', 'alpha_beta'])
df1['beta'] = condition.astype(int)

print(df1[df1.sheet == 'normal'].MCV.min())
print(df1[df1.sheet == 'normal'].MCH.min())

85.0
28.0


In [3]:
df1

,SLHC,HST,HCT,MCV,MCH,MCHC,RDWCV,SLTC,SLBC,FE,FERRITIN,HBA1,HBA2,HBE,HBF,sheet,alpha,beta
PID,,,,,,,,,,,,,,,,,,
97500602,3.02,81.0,0.293,97.1,26.9,277.0,17.0,195.0,7.55,28.0,67.1,96.0,1,NaN,NaN,alpha,1,0
97220957,4.11,123.0,0.399,97.1,29.9,308.0,13.6,214.0,9.56,NaN,NaN,98.3,1.7,NaN,NaN,alpha,1,0
97231143,5.21,141.0,0.495,94.9,27.1,286.0,13.5,339.0,6.54,NaN,NaN,97.8,2.2,NaN,NaN,alpha,1,0
97221041,4.42,133.0,0.418,94.5,30.2,319.0,14.1,165.0,8.13,NaN,NaN,97.8,2.2,NaN,NaN,alpha,1,0
97220964,4.91,146.0,0.463,94.3,29.7,315.0,14.2,270.0,8.18,19.6,1202.1,97.9,2.1,NaN,NaN,alpha,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97501030,6.41,92.0,0.329,51.3,14.3,279.0,24.0,164.0,7.35,10.5,817.4,8.2,2.1,73.6,16.1,alpha_beta,1,1
97220065,5.52,78.0,0.277,50.1,14.1,281.0,22.8,409.0,8.7,12.5,328.87,6.5,10.1,74.4,9.0,alpha_beta,1,1
97501027,6.45,88.0,0.320,49.6,13.6,274.0,22.0,348.0,4.16,23.5,314.0,7.5,10.4,76,6.1,alpha_beta,1,1


In [4]:
def get_df(file_path, sheet_type):
    df = pd.read_csv(file_path)
    df.set_index('PID', inplace=True)

    return df.assign(sheet=sheet_type)

In [5]:
df2 = get_df("../datasets/0504.csv", '0504')
df3 = get_df("../datasets/1904.csv", '1904')
df4 = get_df("../datasets/2704.csv", '2704')
df5 = get_df("../datasets/2804.csv", '2804')
df6 = get_df("../datasets/2904.csv", '2904')

df_arr = [df1, df2, df3, df4, df5, df6]
for e in df_arr:
    print(e.shape)

(19108, 18)
(1671, 18)
(663, 18)
(772, 18)
(664, 18)
(264, 18)


In [6]:
def normal_hc(df):
    # Xác định các trường hợp là normal
    conditions = (df.alpha + df.beta == 0) & (df.MCV >= 85) &(df.MCH >= 28)
    idxes = conditions.index
    return conditions.sum(), idxes, conditions

def both_ab(df):
    # Xác định các trường hợp có cả gen alpha và gen beta xuất hiện đồng thời
    conditions = (df.alpha + df.beta == 2)
    idxes = conditions.index
    return conditions.sum(), idxes, conditions

def alpha(df):
    # Xác định các trường hợp có gen alpha
    conditions = df.alpha == 1
    idxes = conditions.index
    return conditions.sum(), idxes, conditions

def beta(df):
    # Xác định các trường hợp có gen beta
    conditions = df.beta == 1
    idxes = conditions.index
    return conditions.sum(), idxes, conditions

def lack_fe(df):
    # Xác định các trường hợp bị thiếu sắt
    conditions = (~df.FERRITIN.isna()) & (df.FERRITIN < 30)
    df_fe = df[conditions]
    
    alpha_num, alpha_idx, alpha_condition = alpha(df_fe)
    beta_num, beta_idx, beta_condition = beta(df_fe)
    
    print('fe_alpha: ', alpha_num)
    print("fe_beta: ", beta_num)
    
    df_fe2 = df_fe[~(alpha_condition | beta_condition)]
    
    return df_fe2.shape[0], df_fe2.index, conditions
    
    
    

def statistic(df):
    df[features2] =  df[features2].apply(pd.to_numeric, errors='coerce', downcast='float')
    # Tính số lượng, idx, điều kiện để xác định các trường hợp normal
    normal_num, normal_idx, normal_conditions = normal_hc(df)
    
    # loại bỏ các trường hợp normal
    dfa = df[~normal_conditions]
    # Tính số lượng, idx, điều kiện để xác định các trường hợp có chứa cả gen alpha và gen beta đồng thời
    ab_num, ab_idx, ab_conditions = both_ab(dfa)
    
    # Loại bỏ các gen alpha và beta đồng thời
    dfb = dfa[~ab_conditions]
    # Tính số lượng, idx, điều kiện để xác định các trường hợp bị thiếu sắt
    fe_num, fe_idx, fe_conditions = lack_fe(dfb)
    
    # Loại bỏ các trường hợp bị thiếu sắt
    dfc = dfb[~fe_conditions]
    # Tính số lượng, idx, điều kiện để xác định các trường hợp chỉ chứa gen alpha
    alpha_num, alpha_idx, alpha_condition = alpha(dfc)
    # Tính số lượng, idx, điều kiện để xác định các trường hợp chỉ chứa gen beta
    beta_num, beta_idx, beta_condition = beta(dfc)
    
    print('normal: ', normal_num)
    print("alpha_beta: ", ab_num)
    print('normal_fe: ', fe_num)
    print('alpha: ', alpha_num)
    print("beta: ", beta_num)
    
    
# for df in df_arr:
#     print("Kích thước của dữ liệu: ", df.shape)
#     statistic(df)
#     print("----------------------------")

In [7]:
df = pd.concat([df1, df2, df3, df4, df5, df6])
df.shape

(23142, 18)

In [8]:
statistic(df)

fe_alpha:  775
fe_beta:  303
normal:  10164
alpha_beta:  1855
normal_fe:  1109
alpha:  4971
beta:  2851


In [9]:
def get_normal(df):
    conditions = (df.alpha + df.beta == 0) & (df.MCV >= 85) &(df.MCH >= 28)
    return df[conditions]


def get_alpha_beta(df):
    conditions = (df.alpha + df.beta == 2)
    return df[conditions]

def get_alpha(df):
    conditions1 = ~((df.alpha + df.beta == 0) & (df.MCV >= 85) &(df.MCH >= 28))
    conditions2 = (df.alpha == 1) & (df.beta == 0)
    conditions3 = (~df.FERRITIN.isna()) & (~df.FE.isna()) & (df.FERRITIN < 30)
    return df[conditions1 & conditions2 & ~conditions3 ]

def get_beta(df):
    conditions1 = ~((df.alpha + df.beta == 0) & (df.MCV >= 85) &(df.MCH >= 28))
    conditions2 = (df.alpha == 0) & (df.beta == 1)
    conditions3 = (~df.FERRITIN.isna()) & (~df.FE.isna()) & (df.FERRITIN < 30)
    return df[conditions1 & conditions2 & ~conditions3 ]
    

def get_normal_fe(df):
    conditions = (~df.FERRITIN.isna()) & (~df.FE.isna()) & (df.FERRITIN < 30) & (df.alpha + df.beta == 0)
    print(df.alpha.unique(), df.beta.unique())
    return df[conditions]

def negative_unnormal(df):
    conditions1 = ~((df.MCV >= 85) &(df.MCH >= 28))
    conditions2 = df.alpha + df.beta == 0
    return df[conditions1 & conditions2]

get_alpha(df).shape, get_beta(df).shape, negative_unnormal(df).shape

((4972, 18), (2851, 18), (2223, 18))

In [10]:
df.assign(thalas = (df.alpha + df.beta > 0).astype(int))

,SLHC,HST,HCT,MCV,MCH,MCHC,RDWCV,SLTC,SLBC,FE,FERRITIN,HBA1,HBA2,HBE,HBF,sheet,alpha,beta,thalas
PID,,,,,,,,,,,,,,,,,,,
97500602,3.02,81.0,0.293,97.099998,26.900000,277.0,17.000000,195.0,7.55,28.0,67.099998,96.000000,1.0,NaN,NaN,alpha,1,0,1
97220957,4.11,123.0,0.399,97.099998,29.900000,308.0,13.600000,214.0,9.56,NaN,NaN,98.300003,1.7,NaN,NaN,alpha,1,0,1
97231143,5.21,141.0,0.495,94.900002,27.100000,286.0,13.500000,339.0,6.54,NaN,NaN,97.800003,2.2,NaN,NaN,alpha,1,0,1
97221041,4.42,133.0,0.418,94.500000,30.200001,319.0,14.100000,165.0,8.13,NaN,NaN,97.800003,2.2,NaN,NaN,alpha,1,0,1
97220964,4.91,146.0,0.463,94.300003,29.700001,315.0,14.200000,270.0,8.18,19.6,1202.099976,97.900002,2.1,NaN,NaN,alpha,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15038717,4.04,107.0,0.341,84.500000,26.500000,314.0,13.400000,267.0,5.34,56.0,NaN,97.699997,2.3,NaN,NaN,2904,1,0,1
15032051,5.26,138.0,0.435,82.599998,26.100000,316.0,15.200000,177.0,16.65,12.5,353.100006,96.900002,3.1,NaN,NaN,2904,0,0,0
15030562,4.45,139.0,0.420,94.199997,31.100000,330.0,13.600000,273.0,4.36,15.0,98.000000,97.500000,2.5,NaN,NaN,2904,0,0,0


In [26]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, RandomForestRegressor

numerical_columns = ["SLHC","HST","HCT","MCV","MCH","MCHC","RDWCV","SLTC","SLBC", "FERRITIN","FE"]

def fit_model(model, df):
#     global numerical_columns
    df = df.apply(pd.to_numeric, errors='coerce')
    df = df[numerical_columns + ["thalas"]].dropna()
    X = df[numerical_columns]
    y = df['thalas']

    # model.partial_fit(X, y)
    model.fit(X, y)
    return model
    

def evaluate(model, df, desc=''):
#     global numerical_columns
    
    df = df.apply(pd.to_numeric, errors='coerce')
    df = df[numerical_columns + ["thalas"]].dropna()
    X = df[numerical_columns]
    y = df['thalas']
    
    preds = np.array(model.predict(X))
    print('---------------------------------------------------')
    print(f"evaluation results {desc}:\n")
    print(classification_report(y, np.where(preds>=0.5, 1, 0)))
    print('---------------------------------------------------')

    # mask = preds != y
    mask = (0.3 <= preds) & (preds <= 0.7)
    print(sum(mask))

    features_df = df[mask][numerical_columns].reset_index(drop=True)
    labels = preds[mask]
    labels = np.where(labels>=0.5, 1, 0)
    # print(labels)
    new_df = pd.concat([features_df, pd.DataFrame({'thalas': labels})], axis=1)

    return new_df, df[preds == y].reset_index(drop=True)

In [27]:
# model = RandomForestRegressor(n_estimators = 100, random_state=42,n_jobs=8)
# model = xgb.XGBClassifier(learning_rate=0.1, max_depth=10, n_estimators=100, n_jobs=-1, random_state=42,use_label_encoder=False, objective="binary:logistic")
# model = GaussianNB()

In [19]:
# beta_df = get_alpha(df)
# chi lay mẫu có chứa gen beta (ko có norrmal, ko có gen alpha)
beta_df = get_beta(df)
alpha_df = negative_unnormal(df)

print(alpha_df.shape, beta_df.shape)

alpha_df = alpha_df.assign(thalas=0)
beta_df = beta_df.assign(thalas=1)

main_df = pd.concat([alpha_df, beta_df])
dropted_df = main_df.loc[main_df[numerical_columns].dropna().index]
dropted_df = dropted_df.loc[dropted_df[numerical_columns].dropna().index]
train_df, test_df = train_test_split(dropted_df, test_size=0.2, stratify=dropted_df.thalas)

# alpha_df.shape, beta_df.shape, main_df.shape, dropted_df.shape, dropted_df[numerical_columns].dropna().shape
print(train_df.shape, test_df.shape)

(2223, 18) (2851, 18)
(3860, 19) (966, 19)


In [20]:
# fit_model(model, train_df)
# # train_df.info()

In [21]:
# _ = evaluate(model, test_df)

# Thử nghiệm gridsearch cho các mô hình machine learning

Các mô hình được thử nghiệm bao gồm có:
1. RandomForest
2. Naive Bayes
3. SVM
4. KNN
5. K-means
6. XGBoost


In [28]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, RandomForestRegressor
import pickle

numerical_columns = ["SLHC","HST","HCT","MCV","MCH","MCHC","RDWCV","SLTC","SLBC", "FERRITIN","FE"]

def fit_model_grid_search(CV_rfc, df):
#     global numerical_columns
    df = df.apply(pd.to_numeric, errors='coerce')
    df = df[numerical_columns + ["thalas"]].dropna()
    X = df[numerical_columns]
    y = df['thalas']

    CV_rfc.fit(X, y)
    
    return CV_rfc

def save_model(model, path_save):
    with open(path_save, 'wb') as file:  
        pickle.dump(model, file)

def load_model(path_model):
    with open(path_model, 'rb') as file:  
        model = pickle.load(file)
    return model

## 1. Random Forest

In [29]:
# Khởi tạo mô hình
rfc = RandomForestClassifier(random_state=42)

parameters = {
    'n_estimators': [4, 6, 9], 
    'max_features': ['log2', 'sqrt','auto'], 
    'criterion': ['entropy', 'gini'],
    'max_depth': [2, 3, 5, 10], 
    'min_samples_split': [2, 3, 5],
    'min_samples_leaf': [1,5,8]
}

CV_rfc = GridSearchCV(estimator=rfc, param_grid=parameters, cv= 5, verbose=10)
# CV_rfc = fit_model_grid_search(CV_rfc, train_df)

model = RandomForestClassifier(max_depth=10, max_features='log2', min_samples_split=5,
                       n_estimators=9, random_state=42)

In [30]:
model = fit_model(model, train_df)
_ = evaluate(model, test_df)

save_model(model, "../models/radom_forest_model.pkl")

---------------------------------------------------
evaluation results :

              precision    recall  f1-score   support

           0       0.97      0.87      0.92       432
           1       0.90      0.98      0.94       516

    accuracy                           0.93       948
   macro avg       0.93      0.92      0.93       948
weighted avg       0.93      0.93      0.93       948

---------------------------------------------------
0


## 2. Naive Bayes

In [32]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB


from sklearn.model_selection import GridSearchCV

params = {
    'alpha': [0.01, 0.1, 0.5, 1.0, 10.0],
}

bernoulli_nb_grid = GridSearchCV(BernoulliNB(), param_grid=params, n_jobs=-1, cv=5, verbose=10)
# CV_rfc = fit_model_grid_search(bernoulli_nb_grid, train_df)

model = BernoulliNB(alpha=0.01)

model = fit_model(model, train_df)
_ = evaluate(model, test_df)



---------------------------------------------------
evaluation results :

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       432
           1       0.54      1.00      0.70       516

    accuracy                           0.54       948
   macro avg       0.27      0.50      0.35       948
weighted avg       0.30      0.54      0.38       948

---------------------------------------------------
0


/home/tuananh/tuananh/thalassemia/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tuananh/tuananh/thalassemia/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tuananh/tuananh/thalassemia/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

In [33]:
gnb_clf = GaussianNB()
parameters = {
    'var_smoothing': [1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10, 1e-11, 1e-12, 1e-13, 1e-14, 1e-15]
}
clf = GridSearchCV(gnb_clf, parameters, cv=5, verbose=10, n_jobs=1)

# CV_rfc = fit_model_grid_search(clf, train_df)
model = GaussianNB(var_smoothing=1e-08)

model = fit_model(model, train_df)
_ = evaluate(model, test_df)

save_model(model, "../models/naive_bayes_model.pkl")

---------------------------------------------------
evaluation results :

              precision    recall  f1-score   support

           0       0.67      0.77      0.72       432
           1       0.78      0.68      0.73       516

    accuracy                           0.72       948
   macro avg       0.73      0.73      0.72       948
weighted avg       0.73      0.72      0.72       948

---------------------------------------------------
0


## 3. SVM

In [34]:
from sklearn import svm

# Create a Support Vector Classifier
svc = svm.SVC()
# Hyperparameter Optimization
parameters = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
]

# Run the grid search
grid_obj = GridSearchCV(svc, parameters, cv=5, verbose=10, n_jobs=1)
# CV_rfc = fit_model_grid_search(grid_obj, train_df)

model = svm.SVC(C=1, gamma=0.001, kernel='rbf')

model = fit_model(model, train_df)
_ = evaluate(model, test_df)

save_model(model, "../models/svm_model.pkl")

---------------------------------------------------
evaluation results :

              precision    recall  f1-score   support

           0       0.96      0.80      0.87       432
           1       0.85      0.97      0.91       516

    accuracy                           0.89       948
   macro avg       0.90      0.88      0.89       948
weighted avg       0.90      0.89      0.89       948

---------------------------------------------------
0


## 4. KNN

In [35]:
from sklearn.neighbors import KNeighborsClassifier
# Create a KNN Classifier
knn = KNeighborsClassifier()

# Hyperparameter Optimization
parameters = {
    'n_neighbors': [3, 4, 5, 10], 
    'weights': ['uniform', 'distance'],
    'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size' : [10, 20, 30, 50]
}

grid_obj = GridSearchCV(knn, parameters, cv=5, verbose=10, n_jobs=8)
# CV_rfc = fit_model_grid_search(grid_obj, train_df)

model = KNeighborsClassifier(algorithm='auto', leaf_size=10, n_neighbors=10, weights='distance')

model = fit_model(model, train_df)
_ = evaluate(model, test_df)

save_model(model, "../models/knn_model.pkl")

---------------------------------------------------
evaluation results :

              precision    recall  f1-score   support

           0       0.90      0.83      0.86       432
           1       0.86      0.93      0.89       516

    accuracy                           0.88       948
   macro avg       0.88      0.88      0.88       948
weighted avg       0.88      0.88      0.88       948

---------------------------------------------------
0


## 5. KMeans

In [36]:
from sklearn.cluster import KMeans
kmeans = KMeans()

# Hyperparameter Optimization
parameters = {
    "n_clusters": range(2, 11)
}

grid_obj = GridSearchCV(kmeans, parameters, cv=5, verbose=10, n_jobs=8)
# CV_rfc = fit_model_grid_search(grid_obj, train_df)

model = kmeans = KMeans(n_clusters=10, random_state=0)

model = fit_model(model, train_df)
_ = evaluate(model, test_df)

save_model(model, "../models/kmeans_model.pkl")

---------------------------------------------------
evaluation results :

              precision    recall  f1-score   support

           0       0.54      0.10      0.17       432
           1       0.55      0.93      0.69       516

    accuracy                           0.55       948
   macro avg       0.55      0.52      0.43       948
weighted avg       0.55      0.55      0.46       948

---------------------------------------------------
0


## 6. XGboost

In [37]:
from xgboost import XGBClassifier

parameters = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.02, 0.01],
    'n_estimators': [10, 20, 50, 70, 90, 100]
}

xgb = XGBClassifier(use_label_encoder=False, objective="binary:logistic")

grid_obj = GridSearchCV(xgb, parameters, cv=5, verbose=10, n_jobs=8)
# CV_rfc = fit_model_grid_search(grid_obj, train_df)

model = XGBClassifier(learning_rate=0.1, max_depth=5, n_estimators= 90, random_state=42,use_label_encoder=False, objective="binary:logistic")

model = fit_model(model, train_df)
_ = evaluate(model, test_df)

save_model(model, "../models/xgboost_model.pkl")

[22:02:24] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
---------------------------------------------------
evaluation results :

              precision    recall  f1-score   support

           0       0.97      0.89      0.93       432
           1       0.91      0.97      0.94       516

    accuracy                           0.93       948
   macro avg       0.94      0.93      0.93       948
weighted avg       0.94      0.93      0.93       948

---------------------------------------------------
0


/home/tuananh/tuananh/thalassemia/env/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
